In [3]:
import os
import numpy as ny
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [4]:
modules = os.listdir("Modules")

In [5]:
modules_new=modules[:-1] 

In [6]:
modules_new

['top_CLA8.v', 'top_CLA16.v', 'top_CLA64.v', 'top_CLA32.v', 'PGGen.v']

In [7]:
top_modules= [mod for mod in modules if mod[0:3]=='top']

In [8]:
top_modules

['top_CLA8.v', 'top_CLA16.v', 'top_CLA64.v', 'top_CLA32.v']

In [7]:
top_modules[0][:-2]

'top_CLA8'

In [ ]:
os.system("vivado -mode batch -source tcl_add.tcl")

In [9]:
for filename in top_modules:
    os.mkdir("Impl/"+filename[:-2])
    os.system("vivado -mode batch -source tcl_run.tcl -tclargs {}".format(filename))

In [393]:
designs = os.listdir("Impl")
file = ["power.txt", "timing.txt","utilization.txt"]
#col = ["power", "timing","utilization"]

In [338]:
designs 

['.ipynb_checkpoints', 'top_CLA8', 'top_CLA16', 'top_CLA64', 'top_CLA32']

In [339]:
designs_new=designs[1:]

In [340]:
designs_new

['top_CLA8', 'top_CLA16', 'top_CLA64', 'top_CLA32']

## mainprgm

In [341]:
tonpdes = []
sigdes=[]
lutldes=[]
slitdes=[]
spdes=[]

dpddes = []

sldes = []
F7Mdes=[]
lutmdes=[]
lutaldes=[]
slltdes=[]
lut1des=[]
lut2des=[]
lut3des=[]
lut4des=[]
lut5des=[]
lut6des=[]
lut7des=[]


for des in designs_new:
    for fl in file:
        with open("Impl/{}/{}".format(des,fl)) as f:
            lines = f.read()
            
        sp = lines.split("\n")
        
        for l in sp:
            if fl == "power.txt":
                if "Total On-Chip Power" in l:
                    tonp = float(l.split()[6])
                    tonpdes.append(tonp)
           
                if "Signals" in l:
                    sig = float(l.split()[3])
                    sigdes.append(sig) 
                    
                if "Slice Logic" in l:
                    slit = float(l.split()[4])
                    slitdes.append(sig)
                    
                if "LUT as Logic" in l:
                    lutl=float(l.split()[5])
                    lutldes.append(lutl)
                    
                if "Static Power" in l:
                    sp=float(l.split()[4])
                    spdes.append(sp)
                    
            if fl == "timing.txt":
                 if " Data Path Delay" in l:
                    dpd=float(l.split()[3][0:-2])
                    dpddes.append(dpd)
            
            if fl == "utilization.txt":       
                if " F7 Muxes" in l:
                     F7M=int(l.split()[4])
                     F7Mdes.append(F7M)
                if "| LUT as Memory" in l:
                    lutm=int(l.split()[5])
                    lutmdes.append(lutm)
                    
                if "| LUT as Logic" in l:
                    lutal=int(l.split()[5])
                    lutaldes.append(lutal)
                    
                if "| Slice LUTs" in l:
                     sllt=int(l.split()[4])
                     slltdes.append(sllt)
                
                if "LUT1" in l:
                    lut1=int(l.split()[3])
                    lut1des.append(lut1)
                    
                if "LUT2" in l:
                    lut2=int(l.split()[3])
                    lut2des.append(lut2)
                    
                if "LUT3" in l:
                    lut3=int(l.split()[3])
                    lut3des.append(lut3)
                    
                if "LUT4" in l:
                    lut4=int(l.split()[3])
                    lut4des.append(lut4)
                    
                if "LUT5" in l:
                    lut5=int(l.split()[3])
                    lut5des.append(lut5)
                    
                if "LUT6" in l:
                    lut6=int(l.split()[3])
                    lut6des.append(lut6)
                    
                if "LUT7" in l:
                    lut7=int(l.split()[3])
                    lut7des.append(lut7)
                    
                 

In [342]:
Logic=[]
for i in range(0,4):
    Logic.append(lutldes[i]+spdes[i])

In [343]:
dcdes=[]
for i in range(0,4):
    dcdes.append(tonpdes[i]-slitdes[i])

In [383]:
lutasmem2=[0]*len(designs_new)
lutaslg2=[0]*len(designs_new)
sltlg2=[0]*len(designs_new)
lutdiff=[0]*len(designs_new)

In [384]:
#result = [ , ,lutldes ,slitdes ,spdes,dpddes  ,sldes  , ,  , , ,   ]
result = [sigdes,Logic,spdes,dcdes,tonpdes,dpddes,F7Mdes,lutmdes,lutaldes,slltdes,lut1des ,lut2des ,lut3des ,lut4des ,lut5des ,lut6des ,lut7des,lutasmem2,lutaslg2,sltlg2,lutdiff]
df = pd.DataFrame(result)
df.columns = designs_new


In [385]:
df1=df.transpose()

In [386]:
results_new=['Signal','Logic','Static Power','DC','Total','DataPath Delay','F7_Muxes','LUT_as_memory','LUT_as_logic','Slice_LUT','LUT1','LUT2','LUT3','LUT4','LUT5','LUT6','LUT7','LUT_as_memory2','LUT_as_logic2','Slice_LUT2','LUTs_difference']

In [387]:
df1.columns=results_new

In [388]:
df1=df1.fillna(0)

In [389]:
df1

,Signal,Logic,Static Power,DC,Total,DataPath Delay,F7_Muxes,LUT_as_memory,LUT_as_logic,Slice_LUT,...,LUT2,LUT3,LUT4,LUT5,LUT6,LUT7,LUT_as_memory2,LUT_as_logic2,Slice_LUT2,LUTs_difference
top_CLA8,0.170,0.257,0.191,3.278,5.002,6.927,0.0,0.0,13.0,13.0,...,1.0,2.0,3.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0
top_CLA16,0.496,0.448,0.256,10.971,11.141,8.335,0.0,0.0,35.0,35.0,...,6.0,4.0,8.0,5.0,22.0,0.0,0.0,0.0,0.0,0.0
top_CLA64,4.872,3.553,2.405,48.756,49.252,16.020,0.0,0.0,233.0,233.0,...,48.0,14.0,25.0,44.0,155.0,0.0,0.0,0.0,0.0,0.0
top_CLA32,1.724,1.021,0.514,18.347,23.219,11.285,0.0,0.0,97.0,97.0,...,26.0,9.0,14.0,15.0,59.0,0.0,0.0,0.0,0.0,0.0


In [390]:
 df1["LUT_as_logic2"]=df1["LUT1"]+df1["LUT2"]+df1["LUT3"]+df1["LUT4"]+df1["LUT5"]+df1["LUT6"]+df1["LUT7"]
df1["LUT_as_memory2"]=df1["LUT_as_memory"]
df1["Slice_LUT2"]=df1["LUT_as_logic2"]
df1["LUTs_difference"]=df1["LUT_as_logic2"]-df1["Slice_LUT"]

In [391]:
#df1.insert(loc=0, column='Metric', value=col)

df1.to_csv("Result/Result.csv",index=None)

In [392]:
df1

,Signal,Logic,Static Power,DC,Total,DataPath Delay,F7_Muxes,LUT_as_memory,LUT_as_logic,Slice_LUT,...,LUT2,LUT3,LUT4,LUT5,LUT6,LUT7,LUT_as_memory2,LUT_as_logic2,Slice_LUT2,LUTs_difference
top_CLA8,0.170,0.257,0.191,3.278,5.002,6.927,0.0,0.0,13.0,13.0,...,1.0,2.0,3.0,3.0,6.0,0.0,0.0,15.0,15.0,2.0
top_CLA16,0.496,0.448,0.256,10.971,11.141,8.335,0.0,0.0,35.0,35.0,...,6.0,4.0,8.0,5.0,22.0,0.0,0.0,45.0,45.0,10.0
top_CLA64,4.872,3.553,2.405,48.756,49.252,16.020,0.0,0.0,233.0,233.0,...,48.0,14.0,25.0,44.0,155.0,0.0,0.0,286.0,286.0,53.0
top_CLA32,1.724,1.021,0.514,18.347,23.219,11.285,0.0,0.0,97.0,97.0,...,26.0,9.0,14.0,15.0,59.0,0.0,0.0,123.0,123.0,26.0
